In [1]:
import numpy as np
batch = 15
dir = "CEAexec\\CEAexec-win\\"
infile = "template.inp"


# **********Step 1: Create CEA input.inp files***********

rang = np.array([[1, 1.1, 1.2, 1.32, 1.48, 1.65, 1.8, 2, 2.25, 2.5, 2.8, 3.1, 3.5, 3.9, 4.4, 4.9, 5.4, 6, 6.7, 7.4, 8.25, 9.1]])
P = rang * np.array([[10**x for x in range(2,8)]]).transpose()
P = np.round(P, 6)
P = P.flatten()
T1 = np.arange(200, 1e3, 5, dtype=int)
T2 = np.arange(1e3, 8e3, 10, dtype=int)
T = np.concatenate((T1, T2))
fnames = []
print(P.tolist())
for i in range(int(np.ceil(len(P)/batch))):
    for j in range(int(np.ceil(len(T)/batch))):
        Pvals = P[i*batch:min(len(P),(i+1)*batch)]
        Tvals = T[j*batch:min(len(T),(j+1)*batch)]
        Pstr = ','.join([str(pi) for pi in np.round(Pvals/1e5,6)])
        Tstr = ','.join([str(ti) for ti in Tvals.tolist()])
        with open(dir+infile, mode='r') as fread:
            contents = fread.read()
        contents = contents.replace("<PYPRES>", Pstr)
        contents = contents.replace("<PYTEMP>", Tstr)
        writename = f"AUTOFILE_P{i}T{j}"
        fnames.append(writename)
        with open(dir+writename+'.inp', mode='w') as fwrite:
            fwrite.write(contents)

[100.0, 110.0, 120.0, 132.0, 148.0, 165.0, 180.0, 200.0, 225.0, 250.0, 280.0, 310.0, 350.0, 390.0, 440.0, 490.0, 540.0, 600.0, 670.0, 740.0, 825.0, 910.0, 1000.0, 1100.0, 1200.0, 1320.0, 1480.0, 1650.0, 1800.0, 2000.0, 2250.0, 2500.0, 2800.0, 3100.0, 3500.0, 3900.0, 4400.0, 4900.0, 5400.0, 6000.0, 6700.0, 7400.0, 8250.0, 9100.0, 10000.0, 11000.0, 12000.0, 13200.0, 14800.0, 16500.0, 18000.0, 20000.0, 22500.0, 25000.0, 28000.0, 31000.0, 35000.0, 39000.0, 44000.0, 49000.0, 54000.0, 60000.0, 67000.0, 74000.0, 82500.0, 91000.0, 100000.0, 110000.0, 120000.0, 132000.0, 148000.0, 165000.0, 180000.0, 200000.0, 225000.0, 250000.0, 280000.0, 310000.0, 350000.0, 390000.0, 440000.0, 490000.0, 540000.0, 600000.0, 670000.0, 740000.0, 825000.0, 910000.0, 1000000.0, 1100000.0, 1200000.0, 1320000.0, 1480000.0, 1650000.0, 1800000.0, 2000000.0, 2250000.0, 2500000.0, 2800000.0, 3100000.0, 3500000.0, 3900000.0, 4400000.0, 4900000.0, 5400000.0, 6000000.0, 6700000.0, 7400000.0, 8250000.0, 9100000.0, 10000000.

In [2]:
# **********Step 2: Run CEA to produce output.out files***********
import os
import subprocess as sp
if not ("CEAexec-win" in os.getcwd()):
    os.chdir("C:/Users/david/OneDrive - Drexel University/Documents/GitHub/CFE/CEAexec/CEAexec-win")
for file in fnames:
    sub = sp.run(["echo", f"{file}", "|", "FCEA2.exe"], stdout=sp.PIPE, stderr=sp.PIPE, shell=True)


In [29]:
# **********Step 3: Read output files into dictionary ***********

thermo = {"P, BAR":[], "T, K":[], "RHO, KG/CU M":[], "H, KJ/KG":[], "U, KJ/KG":[], 
          "G, KJ/KG":[], "S, KJ/(KG)(K)":[], "M, (1/n)":[], "GAMMAs":[], "SON VEL,M/SEC":[], "VISC,MILLIPOISE":[]}

thermo_conv = {"P, BAR":1e5, "T, K":1, "RHO, KG/CU M":1, "H, KJ/KG":1e3, "U, KJ/KG":1e3, 
          "G, KJ/KG":1e3, "S, KJ/(KG)(K)":1e3, "M, (1/n)":1, "GAMMAs":1, "SON VEL,M/SEC":1, "VISC,MILLIPOISE":1e-4}
frozenheader = "WITH FROZEN REACTIONS"
frozenend = "THERMODYNAMIC PROPERTIES"
frozen = False
transp = {"Cp, KJ/(KG)(K)":[], "CONDUCTIVITY   ":[],"PRANDTL NUMBER":[]}
transp_conv = {"Cp, KJ/(KG)(K)":1e3, "CONDUCTIVITY   ":1e-1,"PRANDTL NUMBER":1}

# iterate over output files
for f in fnames:

    # open each file
    with open(f+".OUT") as file:
        lines = file.readlines()

    # iterate over each line of the file
    for num in range(len(lines)):
        textline = lines[num]
        if frozenheader in textline:
            frozen = True
        elif frozenend in textline:
            #print("stopped")
            frozen = False

        # go through thermodynamic properties
        for dictkey, dictvalue in thermo.items():
            # Check if the property is references in that line
            if dictkey in textline:
                if len(textline)>16:
                    numtext = textline[16:]
                    columns=[]
                    for i in range(0,len(numtext)-8,9):
                        columns.append(numtext[i:i+9])
                else:
                    continue
                # start 
                for property_text in columns:
                    if property_text[-2]=='-' or property_text[-2]==' ':
                        propval = float(property_text[:-2])*10**float(property_text[-2:])
                    else:
                        propval = float(property_text)
                    dictvalue.append(propval * thermo_conv[dictkey])
                    

        if frozen:
            for dictkey, dictvalue in transp.items():
                # Check if the property is references in that line
                if dictkey in textline:
                    if len(textline)>16:
                        numtext = textline[16:]
                        columns=[]
                        for i in range(0,len(numtext)-8,9):
                            columns.append(numtext[i:i+9])
                    else:
                        continue
                    # start 
                    for property_text in columns:
                        if "****" in property_text:
                            print(textline)
                        if property_text[-2]=='-' or property_text[-2]==' ':
                            propval = float(property_text[:-2])*10**float(property_text[-2:])
                        else:
                            propval = float(property_text)
                            if dictkey=="Cp, KJ/(KG)(K)" and propval>100:
                                print(num, frozen, textline, propval, f)
                        dictvalue.append(propval * transp_conv[dictkey])
        
#print(max(transp["Cp, KJ/(KG)(K)"]))



In [31]:
# **********Step 4: Organize dictionary into arrays and write to CSVs ***********
import csv
thermo['VOLUME'] = 1/np.array(thermo["RHO, KG/CU M"])
def writecsv(dictionary):
    # set up first row and first column containing pressure and temperature values
    col1 = np.zeros((len(T),1))
    col1[:,0]=T
    row1 = np.zeros((1,len(P)+1))
    row1[0,1:]=P
    print("IN FUNC")
    # loop through each property
    for key in dictionary.keys():
        # create an empty array for property
        tab = np.zeros((len(T), len(P)))
        tab[:] = np.nan
        # do not generate pressure and temperature tables as these serve as our axes
        if key!="P, BAR" and key!="T, K":
            name = key.split(",")[0].strip().replace(" ", "_")
            filt = filter(lambda s: str.isalpha(s) or s=="_", name)
            csv_fname = ''.join(filt) + ".csv"
            # loop through every value of the property
            for i in range(len(dictionary["P, BAR"])):
                #print("FOUND")
                P_idx = np.where(P==np.round(dictionary["P, BAR"][i],4))
                T_idx = np.where(T==dictionary["T, K"][i])
                tab[T_idx, P_idx] = dictionary[key][i]
            print(tab.shape)
            tab = np.concatenate((col1,tab), axis=1)
            tab = np.concatenate((row1,tab))
            print(csv_fname)
            with open(csv_fname, mode='w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(tab)

# add pressure and temperature values to transport dictionary
transp["P, BAR"] =thermo["P, BAR"]
transp["T, K"] = thermo["T, K"]
#writecsv(thermo)
writecsv(transp)
    
    #Ptable = [[P[i*nT+j] for j in range(nP)] for i in range(nT)]
    #table = lambda x: np.array([[x[i*nT+j] for j in range(nP)] for i in range(nT)])


IN FUNC
(860, 132)
Cp.csv
(860, 132)
CONDUCTIVITY.csv
(860, 132)
PRANDTL_NUMBER.csv


In [32]:
print(P)

[1.00e+02 1.10e+02 1.20e+02 1.32e+02 1.48e+02 1.65e+02 1.80e+02 2.00e+02
 2.25e+02 2.50e+02 2.80e+02 3.10e+02 3.50e+02 3.90e+02 4.40e+02 4.90e+02
 5.40e+02 6.00e+02 6.70e+02 7.40e+02 8.25e+02 9.10e+02 1.00e+03 1.10e+03
 1.20e+03 1.32e+03 1.48e+03 1.65e+03 1.80e+03 2.00e+03 2.25e+03 2.50e+03
 2.80e+03 3.10e+03 3.50e+03 3.90e+03 4.40e+03 4.90e+03 5.40e+03 6.00e+03
 6.70e+03 7.40e+03 8.25e+03 9.10e+03 1.00e+04 1.10e+04 1.20e+04 1.32e+04
 1.48e+04 1.65e+04 1.80e+04 2.00e+04 2.25e+04 2.50e+04 2.80e+04 3.10e+04
 3.50e+04 3.90e+04 4.40e+04 4.90e+04 5.40e+04 6.00e+04 6.70e+04 7.40e+04
 8.25e+04 9.10e+04 1.00e+05 1.10e+05 1.20e+05 1.32e+05 1.48e+05 1.65e+05
 1.80e+05 2.00e+05 2.25e+05 2.50e+05 2.80e+05 3.10e+05 3.50e+05 3.90e+05
 4.40e+05 4.90e+05 5.40e+05 6.00e+05 6.70e+05 7.40e+05 8.25e+05 9.10e+05
 1.00e+06 1.10e+06 1.20e+06 1.32e+06 1.48e+06 1.65e+06 1.80e+06 2.00e+06
 2.25e+06 2.50e+06 2.80e+06 3.10e+06 3.50e+06 3.90e+06 4.40e+06 4.90e+06
 5.40e+06 6.00e+06 6.70e+06 7.40e+06 8.25e+06 9.10e

In [33]:
# ************************ Step 5 (Optional): Delete CEA files ************************
delete = False
if delete:
    for file in os.listdir():
        if "AUTOFILE_" in file:
            os.remove(file)
            